In [79]:
import fastbook
from fastbook import *
import tweepy
import logging
import os
import json
from IPython.display import Image
from urllib.request import urlopen
from PIL import Image
import time

In [90]:
path = Path()
path.ls(file_exts='.pkl')

(#1) [Path('export.pkl')]

In [81]:
learn_inf = load_learner(path/'export.pkl')

In [91]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

def create_api(credentials):
    CONSUMER_KEY = credentials[0]
    CONSUMER_SECRET = credentials[1]
    ACCESS_TOKEN = credentials[2]
    ACCESS_TOKEN_SECRET = credentials[3]
    
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
    api = tweepy.API(auth, wait_on_rate_limit=True, 
        wait_on_rate_limit_notify=True)
    try:
        api.verify_credentials()
    except Exception as e:
        logger.error("Error creating API", exc_info=True)
        raise e
    logger.info("API created")
    return api

In [143]:
def predict_img(img):
    if img == None:
        return None
    else:
        pred,pred_idx,probs = learn_inf.predict(img)
        if pred == "memes":
            pred = "Beep ... esa imagen es bostera ...beep... en un "
        else:
            pred = "Mmm ...beep... esa imagen no parece bostera ...beep... estoy seguro un "
        output = str(f'{pred}{probs[pred_idx]*100:.2f}%')
        return output

In [142]:
def check_mentions(api, since_id):
    logger.info("Retrieving mentions")
    new_since_id = since_id
    for tweet in tweepy.Cursor(api.mentions_timeline,since_id=since_id).items():
        new_since_id = max(tweet.id, new_since_id)
        if tweet.in_reply_to_status_id is not None:
            continue
        if "media" not in tweet.entities:
            continue
        if tweet.entities['media'][0]["type"] == "photo":
            logger.info(f"Answering to {tweet.user.name}")
            
            # Obtengo la URL de la imagen
            image_url = tweet.entities["media"][0]["media_url"]
            # Saco la imagen con PIL
            img = PILImage.create(urlopen(image_url))
            # Obtengo la predicción
            status = predict_img(img)
            
            #Respondo al tweet
            api.update_status(
            status, #ACA VA LA PREDICCIÓN
            in_reply_to_status_id=tweet.id,
            auto_populate_reply_metadata=True
            )
    return new_since_id

In [144]:
def main(credentials):
    api = create_api(credentials)
    since_id = 1
    while True:
        since_id = check_mentions(api, since_id)
        logger.info("Waiting...")
        time.sleep(60)

In [125]:
#enter the corresponding information from your Twitter application management:
CONSUMER_KEY = 'n2NtYdAVq3YTS3iPNYNroot75' #keep the quotes, replace this with your consumer key
CONSUMER_SECRET = 'OY4KOC0Wz8lDMg4jbvBMmx6qrrmqBfXuiySK7OiiPUA1mfkN5p' #keep the quotes, replace this with your consumer secret key
ACCESS_TOKEN = '1420207912471965700-95oUGc5eigK7sRemuBnIdAERf38ed8' #keep the quotes, replace this with your access token
ACCESS_SECRET = 'jYOPKmzGWmdCPvMmwtpxDxezZNZBXQCbvdI5JTN2CiSqu' #keep the quotes, replace this with your access token secret

credentials = [CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET]

In [126]:
if __name__ == "__main__":
    main(credentials)

INFO:root:API created
INFO:root:Retrieving mentions
INFO:root:Answering to Erik Otero Chio


INFO:root:Answering to Erik Otero Chio


INFO:root:Answering to Erik Otero Chio


INFO:root:Waiting...
INFO:root:Retrieving mentions
INFO:root:Waiting...


KeyboardInterrupt: 

In [139]:
# Obtengo la URL de la imagen
image_url = "https://pbs.twimg.com/media/E8KPoGRWQAIFi0f?format=jpg&name=small"
# Saco la imagen con PIL
img = PILImage.create(urlopen(image_url))

if img == None:
    print(None)
else:
    pred,pred_idx,probs = learn_inf.predict(img)
    if pred == "memes":
        pred = "Beep ... ese meme es bostero ...beep... en un "
    else:
        pred = "Mmm ...beep... ese meme no parece bostero ...beep... estoy seguro un "
    output = str(f'{pred}{probs[pred_idx]*100:.2f}%')
    print(output)

Beep ... ese meme es bostero ...beep... en un 99.47%
